<center><h1>VC10: Page Rank</h1></center>

# RECUERDA RELLENAR TUS DATOS A CONTINUACIÓN ANTES DE HACER NADA

In [7]:
# ===============================================================#
# Rellena AQUÍ tu nombre y apellidos antes de hacer nada
# ===============================================================#

NOMBRE = 'CarlosEsteban'
APELLIDOS = 'PosadaMejia'

# ===============================================================#
# NO MODIFIQUES ESTA PORCIÓN DE CÓDIGO, ES PARA LA EVALUACIÓN
# MUY IMPORTANTE: NO MODIFICAR
# ===============================================================#
from matplotlib.backends.backend_agg import FigureCanvasAgg
def encode_figure(fig):
    canvas = FigureCanvasAgg(fig)
    canvas.draw()
    img, (width, height) = canvas.print_to_buffer()
    return {'img': img, 'width': width, 'height': height}

answers = {}
answers['name'] = NOMBRE
answers['surname'] = APELLIDOS
answers['subject'] = '06MAIR_10_A_2020-21_ANS'
answers['ex'] = 'N4.1'
# ===============================================================#

En este tutorial vamos a ver las diferentes maneras de calcular 
el PageRank de un grafo. Para ello, vamos a usar el siguiente 
grafo de ejemplo:


<img src="images/grafo.png" style="width:300px"/>

Una manera habitual de representar un grafo en computación es 
a través de su matriz de adyacencia. Una matriz de adyacencias 
es una matriz cuadrada de tantas filas/columnas como nodos hay 
en el grafo que codifica las posibles transiciones: un uno en 
la celda $(i,j)$ señala que hay un arco (dirigido) desde el nodo 
$i$ al nodo $j$. En este caso, dicha matriz sería:

<pre>
+---------------+
| 0  0  0  1  1 |
| 1  0  1  0  0 |
| 0  0  0  0  1 |
| 0  1  1  0  0 |
| 0  0  1  1  0 |
+---------------+
</pre>

El producto matricial de la matriz de adyacencias consigo misma 
devuelve una nueva matriz que indica los nodos comunicados por 
caminos de tamaño dos. En este sentido, la potencia de la matriz 
elevada a $l$ devuelve una matriz que indica si existen caminos 
de longitud $l$ entre cada par de nodos:


In [8]:
import numpy as np

D = np.array([[0,1,0,0,0],[0,0,0,1,0],[0,1,0,1,1],[1,0,0,0,1],[1,0,1,0,0]]).T

print("Matriz original:")
print(D)
print("Matriz elevada a la potencia 3 (al cubo):")
print(np.linalg.matrix_power(D,3))


Matriz original:
[[0 0 0 1 1]
 [1 0 1 0 0]
 [0 0 0 0 1]
 [0 1 1 0 0]
 [0 0 1 1 0]]
Matriz elevada a la potencia 3 (al cubo):
[[1 1 2 0 2]
 [0 1 3 2 0]
 [0 1 1 0 1]
 [0 0 1 2 2]
 [1 0 2 1 1]]



Esta última matriz indica que hay, p.ej., 3 posibles caminos que 
van desde el segundo nodo al tercero. 


El cálculo del algoritmo PageRank se basa en el uso de un grafo bien 
conectado. Podemos saberlo haciendo el siguiente cálculo. 
Si después de la suma de las primeras X potencias de la 
matriz todavía hay algún cero en la matriz resultante, 
el grafo <b>no</b> es fuertemente conexo. 

In [10]:
B = np.diag(np.ones(D.shape[0]))
prod = D
for i in np.arange(1,5):
    B += np.linalg.matrix_power(D,i)

print("Número de caminos posibles en un máximo de 4 pasos entre cualquier par de nodos:")
print(B)

Número de caminos posibles en un máximo de 4 pasos entre cualquier par de nodos:
[[3. 2. 7. 5. 6.]
 [2. 4. 7. 3. 5.]
 [1. 1. 5. 2. 3.]
 [1. 3. 7. 5. 4.]
 [1. 2. 6. 4. 6.]]



El PageRank no se calcula sobre la matriz de adyacencias, sino 
que se calcula con la matriz de transiciones. La <b>matriz de 
transiciones</b> indica la probabilidad de moverse de un nodo a 
otro. Así, el valor en la celda 
$(i,j)$ indica la probabilidad de que saliendo del nodo $j$ 
lleguemos al $i$. Si asumimos igual probabilidad en todas las 
transiciones, la probabilidad de moverse de $j$ a $i$ es igual 
a 1 entre el <i>outdegree</i> de $j$ (número de arcos que salen 
del nodo $j$). Intuitivamente, la suma de las columnas es igual 
a 1. Sólo será 0 si el nodo correspondiente no tiene arcos 
de salida (<i>outdegree</i> $=0$). De acuerdo con esta definición, 
la matriz de transiciones del grafo anterior sería:


In [11]:
outdegree = np.sum(D,axis=1)
A = (D / outdegree[:,np.newaxis]).T

print("La matriz de transiciones de D es:")
print(A)

La matriz de transiciones de D es:
[[0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.5 0. ]
 [0.  0.5 0.  0.5 0.5]
 [0.5 0.  0.  0.  0.5]
 [0.5 0.  1.  0.  0. ]]


Disponiendo de esta matriz, el PageRank se calcula como la 
potencia $n$-ésima de la matriz de transiciones por un vector 
de inicio uniformemente aleatorio (todos los valores son iguales a 1 entre 
el número de nodos) indicando que todas los nodos tienen 
inicialmente la misma probabilidad de ser el primer nodo en ser visitado.

In [12]:
# F: Vector de inicio aleatorio
F = np.ones(D.shape[0])
F /= np.sum(F)

# Cálculo del page rank para diferentes potencias
for i in np.arange(1,31):
    rank = np.dot(np.linalg.matrix_power(A,i),F)
    print("PageRank para A**{:2d} es: {}".format(i,rank))

PageRank para A** 1 es: [0.1 0.1 0.3 0.2 0.3]
PageRank para A** 2 es: [0.05 0.1  0.3  0.2  0.35]
PageRank para A** 3 es: [0.05  0.1   0.325 0.2   0.325]
PageRank para A** 4 es: [0.05   0.1    0.3125 0.1875 0.35  ]
PageRank para A** 5 es: [0.05    0.09375 0.31875 0.2     0.3375 ]
PageRank para A** 6 es: [0.046875 0.1      0.315625 0.19375  0.34375 ]
PageRank para A** 7 es: [0.05      0.096875  0.31875   0.1953125 0.3390625]
PageRank para A** 8 es: [0.0484375  0.09765625 0.315625   0.19453125 0.34375   ]
PageRank para A** 9 es: [0.04882812 0.09726563 0.31796875 0.19609375 0.33984375]
PageRank para A**10 es: [0.04863281 0.09804688 0.31660156 0.19433594 0.34238281]
PageRank para A**11 es: [0.04902344 0.09716797 0.31738281 0.19550781 0.34091797]
PageRank para A**12 es: [0.04858398 0.09775391 0.31679688 0.1949707  0.34189453]
PageRank para A**13 es: [0.04887695 0.09748535 0.31730957 0.19523926 0.34108887]
PageRank para A**14 es: [0.04874268 0.09761963 0.31690674 0.19498291 0.34174805]
PageRa


Se observa claramente que a medida que se aumenta el valor del exponente (potencia), el resultado del PageRank que se calcula es más estable.

El PageRank también se puede calcular como el primer vector propio de la matriz de transiciones normalizado:

In [13]:
_, vects = np.linalg.eig(A)
aux = vects[:,0]
aux /= np.sum(aux)
print('PageRank calculado mediante técnicas algebraicas:',aux.real)

PageRank calculado mediante técnicas algebraicas: [0.04878049 0.09756098 0.31707317 0.19512195 0.34146341]


Se puede ver que devuelven el mismo valor, aunque el cálculo 
con potencias es aproximado. Si aumentamos o reducimos el 
número del exponente en la potencia, la aproximación será más o menos fiable.

Y hasta aquí, el cálculo del PageRank. Vamos a hacer ahora 
una prueba usando otro grafo diferente:

<img src="images/grafo_sinDtoB.png" style="width:300px" />

que se representa por medio de la siguiente matriz de adyacencias:

In [14]:
D = np.array([[0,1,0,0,0],[0,0,0,0,0],[0,1,0,1,1],[1,0,0,0,1],[1,0,1,0,0]]).T

print("Matriz de adyacencias del segundo grafo:")
print(D)

outdegree = np.sum(D,axis=1)
A = (D / outdegree[:,np.newaxis]).T

print("Matriz de transiciones del segundo grafo:")
print(A)

rank = np.dot(np.linalg.matrix_power(A,30),F)
print("PageRank para A**{:2d} es: {}".format(30,rank))

Matriz de adyacencias del segundo grafo:
[[0 0 0 1 1]
 [1 0 1 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [0 0 1 1 0]]
Matriz de transiciones del segundo grafo:
[[0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.5 0.  1.  0.5]
 [0.5 0.  0.  0.  0.5]
 [0.5 0.  1.  0.  0. ]]
PageRank para A**30 es: [0.         0.         0.40000305 0.2        0.39999695]



Observen que este grafo no tiene arcos que apunten al nodo $B$. 
Esto genera problemas en los cálculos del PageRank: si no hay 
ningún nodo apuntando a $B$, éste obtiene un <i>score</i> (valor 
de PageRank) igual a $0$. También recibe valor $0$ cualquier otro 
nodo que sólo está apuntado por nodos del estilo de $B$. 
Esto no es realista, pues todas los 
nodos son alcanzables (¡al menos por quien lo creó!). Los problemas 
que este hecho acarrea son abordados mediante una modificación 
del algoritmo original. Ahora, además del inicio aleatorio, se 
asumirá que, con cierta probabilidad ($p$), el camino puede 
realizar un salto aleatorio y seguir en cualquier otro nodo 
del grafo (aleatoriamente) aunque no estén directamente conectados. 

En la práctica, se construye una matriz de transiciones alternativa $M$ 
que toma los valores de la matriz de transiciones original $A$ con 
probabilidad $(1-p)$ y, con probabilidad $p$, los de otra matriz 
de transiciones aleatorias $R$ (se puede saltar de cualquier nodo 
a cualquier otro con la misma probabilidad, $1/|V|$, 
donde $|V|$ es el número de nodos; $|V|=5$ en este caso).


In [17]:
p = 0.15
R = np.ones(D.shape)
R /= D.shape[0]

M = (1-p)*A + p*R
print("La matriz de transiciones original, A, es:")
print(A)
print("La matriz de transiciones alternativa, M, es:")
print(M)

La matriz de transiciones original, A, es:
[[0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.5 0.  1.  0.5]
 [0.5 0.  0.  0.  0.5]
 [0.5 0.  1.  0.  0. ]]
La matriz de transiciones alternativa, M, es:
[[0.03  0.455 0.03  0.03  0.03 ]
 [0.03  0.03  0.03  0.03  0.03 ]
 [0.03  0.455 0.03  0.88  0.455]
 [0.455 0.03  0.03  0.03  0.455]
 [0.455 0.03  0.88  0.03  0.03 ]]



Como puede observarse, ahora existe la posibilidad de desplazarse 
de cualquier nodo a cualquier otro en un único paso.

Una vez obtenida la nueva matriz de transiciones, $M$, podemos 
usarla para calcular el PageRank, como se vio anteriormente, 
cambiando únicamente la matriz $A$ por $M$:


In [20]:
F = np.ones(D.shape[0])
F /= np.sum(F)

rank = np.dot(np.linalg.matrix_power(M,30),F)
print("PageRank aproximado para M**{:2d} es: {}".format(30,rank))

_, vects = np.linalg.eig(M)
rank = vects[:,0]
rank /=np.sum(rank)
print("PageRank (algebra) para M es: {}".format(rank.real))

PageRank aproximado para M**30 es: [0.04275    0.03       0.36650801 0.20104148 0.35970052]
PageRank (algebra) para M es: [0.04275    0.03       0.36650798 0.20104148 0.35970054]



<hr />

<center><h1>Algoritmo HITS</h1></center>

El algoritmo HITS, como hemos visto, es similar en concepción 
a PageRank. En ese sentido, los cálculos también tienen semejanzas.
Podemos usar dos estrategias igualmente: multiplicación de matrices 
o vectores propios. La diferencia principal es que éste no usa 
la matriz de transiciones sino la de adyacencias. Así, el cálculo 
de HITS con multiplicación de matrices para el grafo usado en 
la sección anterior es:


In [23]:
D = np.array([[0,1,0,0,0],[0,0,0,1,0],[0,1,0,1,1],[1,0,0,0,1],[1,0,1,0,0]]).T

h = np.ones(D.shape[0])
a = np.array([])

for i in np.arange(30):
    a = np.dot(D.T, h)
    h = np.dot(D, a)## TU CODIGO AQUI ##)

print("El valor de autoridad (A) es  : {}".format(a/np.sum(a)))
print("El valor de centralidad (H) es: {}".format(h/np.sum(h)))


# INICIO - NO TOCAR : necesario para la evaluación del notebook
answers['1'] = {'a': a,
                'h': h}
# FIN - NO TOCAR : necesario para la evaluación del notebook

El valor de autoridad (A) es  : [0.12711028 0.12711028 0.40658667 0.23315152 0.10604124]
El valor de centralidad (H) es: [0.15759064 0.24795826 0.04926729 0.24795826 0.29722555]



Los vectores de <b>authorities</b> (autoridad) y <b>hubs</b> 
(centralidad) se obtienen 
de manera iterativa e intercambiándose mútuamente para calcular 
el otro. La segunda estrategia implica el uso del cálculo de los 
vectores propios de dos matrices diferentes: la matriz obtenida 
de multiplicar 
la transformada de $D$ por $D$; y la matriz obtenida de 
multiplicar $D$ por la transformada de $D$. Es necesario 
recordar que la multiplicación matricial <b>no</b> es conmutativa. 


In [24]:
vals, vects = np.linalg.eig(np.dot(D.T,D))
a = vects[:,np.argmax(vals)]
print("El valor de autoridad (A) es  : {}".format(a/np.sum(a)))

vals, vects = np.linalg.eig(np.dot(D,D.T))
h = vects[:,np.argmax(vals)]
print("El valor de centralidad (H) es: {}".format(h/np.sum(h)))

El valor de autoridad (A) es  : [0.12711039 0.12711039 0.40658689 0.23315136 0.10604097]
El valor de centralidad (H) es: [0.15759045 0.24795841 0.04926716 0.24795841 0.29722557]


Como también vimos con PageRank, el cálculo con la multiplicación 
de matrices es aproximado. Según aumentemos el número de iteraciones 
del bucle obtendremos una aproximación más precisa.


<hr />

<center><h1>Personalized PageRank</h1></center>

Para el estudio del algoritmo <i>Personalized 
PageRank</i>, usaremos un nuevo grafo:

<img src="images/grafo_ppr.png" style="width:300px" />

que se codifica mediante la siguiente matriz de adyacencias:


In [25]:
D = np.zeros((16,16))
D[0,1] = D[1,0] = 1; D[1,2] = D[2,1] = 1
D[1,3] = D[3,1] = 1; D[1,3] = D[3,1] = 1
D[2,4] = D[4,2] = 1; D[2,5] = D[5,2] = 1
D[2,6] = D[6,2] = 1; D[2,7] = D[7,2] = 1
D[5,6] = D[6,5] = 1; D[6,7] = D[7,6] = 1
D[3,7] = D[7,3] = 1; D[4,8] = D[8,4] = 1
D[4,9] = D[9,4] = 1; D[4,10] = D[10,4] = 1
D[9,10] = D[10,9] = 1; D[5,10] = D[10,5] = 1
D[5,11] = D[11,5] = 1; D[10,11] = D[11,10] = 1
D[7,12] = D[12,7] = 1; D[7,13] = D[13,7] = 1
D[14,11] = D[11,14] = 1; D[14,12] = D[12,14] = 1
D[14,15] = D[15,14] = 1;

print("La matriz de adyacencias D es:")
print(D.astype(int))

A = D.T/np.sum(D,axis=1)

print("La matriz de transiciones A es:")
print(np.round(A,2))


La matriz de adyacencias D es:
[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0]
 [0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 1 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]]
La matriz de transiciones A es:
[[0.   0.33 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [1.   0.   0.2  0.5  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.33 0.   0.   0.25 0.25 0.33 0.2  0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.33 0.   0.   0.   0.   0.   0.2  0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.2  0.   0.   0.   0.   0.   1.


Similar al PageRank normal, existen tres maneras de calcular el PPR: 
con potencias 
de matrices, vectores propios o muestreo. En el primer 
caso, la potencia de la matriz de transiciones adaptada ($M$) 
se obtiene:


In [26]:
p = 0.15

R = np.zeros(D.shape)
R[0,:] = 1
F = R[:,0]

M = (1-p)*A + p*R

rank = np.dot(np.linalg.matrix_power(M,30),F)
print("Personalized PageRank aproximado para M**{:2d} es: \n{}".format(30,rank))

Personalized PageRank aproximado para M**30 es: 
[0.21788325 0.23950534 0.11263876 0.08286386 0.03853373 0.04367079
 0.04340574 0.08800673 0.00819452 0.01448182 0.02958747 0.02105317
 0.02045852 0.0149806  0.01925777 0.00547792]



En este caso, el nodo inicial es sólo el nodo 1. Le 
asignamos probabilidad 1 (y 0 al resto) para indicar 
que todos los caminos empiezan (y se reinician) siembre 
desde ese nodo en el grafo. 
Podríamos repartir la probabilidad de inicio entre 
varios nodos. Nótese que si repartimos la probabilidad 
de manera uniforme entre todos los nodos nos encontraremos 
en el PageRank tradicional.

La segunda manera de calcular el Personalized PageRank es usando la técnica basada en álgebra:


In [28]:
_, vects = np.linalg.eig(M)
rank = vects[:,0]
rank /=np.sum(rank)
print("Personalized PageRank (algebra) para M es: \n{}".format(rank.real))

Personalized PageRank (algebra) para M es: 
[0.21786936 0.23953892 0.11260915 0.08283861 0.03854741 0.04368739
 0.04339638 0.08805442 0.00819132 0.01447899 0.02958899 0.02103716
 0.02043518 0.01496925 0.01929152 0.00546593]



Finalmente, podemos usar el muestreo. Básicamente se trata 
de lanzar $n$ <i>random walks</i> de longitud $l$. Dejamos 
que cada camino complete los $l$ pasos y guardamos el 
nodo en el que acaba cada uno de los caminos. Es necesario 
recordar que, con cierta probabilidad ($p$), el camino puede 
volver al nodo inicial para reiniciar un camino que realizará 
sólo los $\hat{l}$ pasos restantes. Al final, la frecuencia con 
la que se llega a esos nodos finales sigue la distribución 
de probabilidad que representa el PageRank:


In [37]:
np.where(D[1, :].flatten()!= 0)[0]

array([0, 2, 3])

In [39]:
np.random.seed(17)

p = 0.15
first_node = 0
nwalks = 3000
walk_len = 20
rank = np.zeros(D.shape[0])
for w in np.arange(nwalks):
    act = first_node
    
    for s in np.arange(walk_len):
        if p > np.random.random_sample():
            act = first_node
        else:
            cand = np.where(D[act,:].flatten() != 0)[0]
            act = np.random.choice(cand)## TU CODIGO AQUI ##
    rank[act] += 1

rank /= np.sum(rank)
print("Personalized PageRank aproximado (muestreo) para D es: \n{}".format(rank))
print("El valor aproximado de Personalized PageRank del primer nodo es:",rank[0])

# INICIO - NO TOCAR : necesario para la evaluación del notebook
answers['2'] = {'act': act,
                'rank': rank}
# FIN - NO TOCAR : necesario para la evaluación del notebook

Personalized PageRank aproximado (muestreo) para D es: 
[0.223      0.23433333 0.12433333 0.08333333 0.046      0.03766667
 0.043      0.07933333 0.00566667 0.01266667 0.03166667 0.01966667
 0.02066667 0.01333333 0.02       0.00533333]
El valor aproximado de Personalized PageRank del primer nodo es: 0.223



El cálculo con la potencia de matrices y el de muestreo 
son ambos aproximados. Según aumentemos el número de iteraciones 
del bucle obtendremos una aproximación más precisa. En el caso 
del muestreo, esto se hace aumentando el número de caminos 
aleatorios (<i>nwalks</i>). El tamaño (<i>walk_len</i>) de 
los caminos debería ser, como mínimo, el suficiente para 
que un camino aleatorio llegue al punto más alejado, aunque 
se puede restringir según la necesidad o el uso.


# RECUERDA EJECUTAR ESTA CELDA ÚNICAMENTE CUANDO HAYAS FINALIZADO EL NOTEBOOK

In [40]:
################################################################################
# ATENCIÓN, MUY IMPORTANTE: EJECUTA ESTA CELDA SIN MODIFICAR NADA!
#
# ESTA CELDA SE ENCARGA DE COMPRIMIR LOS RESULTADOS OBTENIDOS Y MANDARLOS
# AL SERVIDOR PARA SU VERIFICACIÓN Y ALMACENAJE.
#
# SI NO LA EJECUTAS, ¡¡¡NO PODRÁS SER EVALUADX!!!
################################################################################

import zmq
import base64
import pickle
import bz2
import sys

def send_results(answers):

    def compress_data(data):
        data_pkl = pickle.dumps(data)
        data_bz2 = bz2.compress(data_pkl)
        return data_bz2

    if NOMBRE == 'TuNombre' or APELLIDOS == 'TusApellidos' or \
    NOMBRE.strip() == '' or APELLIDOS.strip() == '':
        print('Rellena tu nombre y apellidos en la primera celda del notebook!')
    else:
        print('Conectando con el servidor de evaluación...', end='\t')
        context = zmq.Context()
        socket = context.socket(zmq.REQ)
        socket.connect(base64.b64decode(b'dGNwOi8vMTU4LjQyLjE3MC4xMzU6MzM4OQ==').decode('ascii'))
        print('OK')

        # Compressing data
        print('Comprimiendo las respuestas...', end='\t')
        data = compress_data(answers)
        print('OK')

        print('Enviando datos...', end='\t')
        socket.send(data)
        message = socket.recv()
        if message == data:
            print(f'OK')
            print('Entrega realizada con éxito! :)')
        else:
            print('ERROR! Por favor vuelve a ejecutar la celda. Si los problemas persisten ponte en contacto con el profesor.')    


send_results(answers)

Conectando con el servidor de evaluación...	OK
Comprimiendo las respuestas...	OK
Enviando datos...	OK
Entrega realizada con éxito! :)
